# Code Challenge

$$sigmoid(z) = \frac{1}{1 + e^{-(z)}}\tag{1}$$  
$$z^{(i)} = w^T x^{(i)} + b \tag{2}$$  
$$\hat{y}^{(i)} = a^{(i)} = sigmoid(z^{(i)})\tag{3}$$  
$$ \mathcal{L}(a^{(i)}, y^{(i)}) =  - y^{(i)}  \log(a^{(i)}) - (1-y^{(i)} )  \log(1-a^{(i)})\tag{4}$$  

The cost is then computed by summing over all training examples:
$$ J = \frac{1}{m} \sum_{i=1}^m \mathcal{L}(a^{(i)}, y^{(i)})\tag{5}$$

## 1.Sigmoid

<hr/>

$$sigmoid(z) = \frac{1}{1 + e^{-(z)}}\tag{1}$$

In [54]:
import numpy as np
def sigmoid(z):
    """
    input:
    z -- scalar or numpy array of any size.

    Output:
    s -- sigmoid(z)
    """

    ### START CODE HERE ### 
    s = 1/(1+np.exp(-z))
    ### END CODE HERE ###
    assert(len(s)==len(z))
    
    return s

In [73]:
s=sigmoid(np.arange(0,10))
answer=np.array([0.5, 0.73105858, 0.88079708, 0.95257413, 0.98201379, 0.99330715,
                 0.99752738, 0.99908895, 0.99966465, 0.99987661])
assert(np.round(s,6)==np.round(answer,6)).all()

print('Sigmoid completed.','\n\nWell Done!!')

Sigmoid completed. 

Well Done!!


## 2.Initialize Parameters

W=Coefficients (weights) | B<sub>1</sub> to B<sub>n</sub>  
b=intercept (bias) | B<sub>0</sub>

In [89]:
def initialize_with_zeros(dim):
    """
    This function creates a vector of zeros of shape (dim, 1) for w and initializes b to 0.
    
    Input:
    dim -- size of the w vector
    
    Output:
    w -- initialized vector of zeros with shape (dim, 1)
    b -- initialized scalar as zero (corresponds to the bias)
    """
    
    ### START CODE HERE ### 
    w = np.zeros((dim,1))
    b = 0
    ### END CODE HERE ###

    assert(w.shape == (dim, 1))
    assert(isinstance(b, float) or isinstance(b, int))
    
    return w, b

In [141]:
w, b=initialize_with_zeros(5)
assert((w==np.array([0,0,0,0,0])).all() & (b==0))
print('Initialize Parameters Completed.','\n\nWell Done!!')

Initialize Parameters Completed. 

Well Done!!


## 3.Forward and Backward propagation

Forward Propagation:  
1. $A = \sigma(w^T X + b) = (a^{(1)}, a^{(2)}, ..., a^{(m-1)}, a^{(m)})$
2. $J = -\frac{1}{m}\sum_{i=1}^{m}y^{(i)}\log(a^{(i)})+(1-y^{(i)})\log(1-a^{(i)})$  

These are the equations after derivation of cross entropy:
$$ dw= \frac{\partial J}{\partial w} = \frac{1}{m}X(A-Y)^T\tag{6}$$
$$ db= \frac{\partial J}{\partial b} = \frac{1}{m} \sum_{i=1}^m (a^{(i)}-y^{(i)})\tag{7}$$

In [139]:
def propagate(w, b, X, Y):
    """
    Implement the cost function and its gradient for the propagation explained above

    Inputs:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of size (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat) of size (1, number of examples)

    Outputs:
    cost -- negative log-likelihood cost for logistic regression
    dw -- gradient of the loss with respect to w, thus same shape as w
    db -- gradient of the loss with respect to b, thus same shape as b
    
    Tips:
    - Write your code step by step for the propagation. np.log(), np.dot()
    """
    
    m = X.shape[1]
    
    # FORWARD PROPAGATION (FROM X TO COST)
    ### START CODE HERE ### (≈ 2 lines of code)
    A = sigmoid(np.dot(w.T,X)+b)                                    # compute activation
    cost = -(1/m)*np.sum((Y*np.log(A)+(1-Y)*np.log(1-A)))           # compute cost
    ### END CODE HERE ###
    
    # BACKWARD PROPAGATION (TO FIND GRAD)
    ### START CODE HERE ### (≈ 2 lines of code)
    dw = (1/m)*np.dot(X,(A-Y).T)
    db = (1/m)*np.sum(A-Y)
    ### END CODE HERE ###

    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

In [142]:
w= np.array([[2.],[3.]])
b=5.
X=np.array([[0,1.,-1.],[3.,4.,-3.2]])
Y=np.array([[0,1,1]])
grads, cost = propagate(w, b, X, Y)


assert((np.round(grads['dw'],6)==np.round(np.array([[0.33288049], [2.06521674]]),6)).all() & (np.round(grads['db'],6)==np.round(0.00045256093986822954,6)))
print('Forward and Backward propagation Completed.','\n\nWell Done!!')

Forward and Backward propagation Completed. 

Well Done!!


## 4. Optimization parameters.

Parameters Update:  
w<sub>new</sub> = w<sub>old</sub> - learning_rate \* dw  
b<sub>new</sub> = b-<sub>old</sub> - learning_rate \* db  

In [144]:
# GRADED FUNCTION: optimize

def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
    """
    This function optimizes w and b by running a gradient descent algorithm
    
    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of shape (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat), of shape (1, number of examples)
    num_iterations -- number of iterations of the optimization loop
    learning_rate -- learning rate of the gradient descent update rule
    print_cost -- True to print the loss every 100 steps
    
    Returns:
    params -- dictionary containing the weights w and bias b
    grads -- dictionary containing the gradients of the weights and bias with respect to the cost function
    costs -- list of all the costs computed during the optimization, this will be used to plot the learning curve.
    
    Tips:
    You basically need to write down two steps and iterate through them:
        1) Calculate the cost and the gradient for the current parameters. Use propagate().
        2) Update the parameters using gradient descent rule for w and b.
    """
    
    costs = []
    
    for i in range(num_iterations):
        
        
        # Cost and gradient calculation (≈ 1-4 lines of code)
        ### START CODE HERE ### 
        grads, cost = propagate(w, b, X, Y)
        ### END CODE HERE ###
        
        # Retrieve derivatives from grads
        dw = grads["dw"]
        db = grads["db"]
        
        # update rule (≈ 2 lines of code)
        ### START CODE HERE ###
        w = w-learning_rate*(dw)
        b = b-learning_rate*(db)
        ### END CODE HERE ###
        
        # Record the costs
        if i % 100 == 0:
            costs.append(cost)
        
        # Print the cost every 100 training iterations
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

In [145]:
params, grads, costs = optimize(w, b, X, Y, num_iterations= 100, learning_rate = 0.009, print_cost = False)
assert()

In [160]:
assert((np.round(params['w'],6)==np.round(np.array([[1.7225749 ], [1.21227121]]),6)).all() & (np.round(params['b'],6)==np.round(4.977436310248131
,6)) & (np.round(costs,6)==np.round([6.867120093527537],6)))



This notebook was made similar to Andrew Ng's homework in coursera.  
Resource:  
please follow the link to check the course(highly recommended): https://www.coursera.org/specializations/deep-learning